In [1]:
import sys
import geopandas as gpd
import matplotlib.pyplot as plt

sys.path.append('../utils')
import data_paths
from traffic_counts import TrafficCounts
from hbefa_hot_emissions import HbefaHotEmissions
from excel_calendar import Calendar

import datetime

from multiprocessing import Pool

import inventory_processing

In [2]:
# import visum model
visum = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg')
visum_red = visum.drop(visum.loc[(visum['road_type'] == 'none') | (visum['dtv_SUM']==0)].index, axis = 0)

visum_red.loc[visum_red['road_type']=='Access-residential', 'road_type'] = 'Local/Collector'
visum_red.loc[visum_red['road_type']=='Distributor/Secondary', 'road_type'] = 'TrunkRoad/Primary-National'

In [3]:
# cut visum model to the Munich area
munich_boarders = gpd.read_file(data_paths.MUNICH_BOARDERS_FILE).to_crs(25832)
visum_red = gpd.clip(visum_red, munich_boarders)

In [4]:
# initialize calendar
cal = Calendar()
# initialize traffic cycles
cycles = TrafficCounts()
# initialize HBEFA emission factors
hbefa = HbefaHotEmissions()

Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_PC.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_LCV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_HGV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_Coach.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_MOT.XLS


In [5]:
# get dates for normweekdays in 2019
dates = cal.get_calendar()
normweekdays_2019 = dates[(dates['day_type'] == 1) &
                          (dates['date'].between('2019-01-01','2019-12-31'))]

# calculate vehicle shares on average norm weekday in 2019
# these values are used to calculate the vehicle share correction factors
weekday_shares_2019 = cycles.vehicle_shares.loc[:,normweekdays_2019['date']].reset_index()
weekday_shares_2019 = weekday_shares_2019.groupby(['road_type','vehicle_class'])[0].mean()

In [6]:
visum_red['hgv_corr'] = visum_red.apply(lambda row : row['delta_HGV'] / weekday_shares_2019.loc[row['road_type'],'HGV'], axis = 1)
visum_red['lcv_corr'] = visum_red.apply(lambda row : row['delta_LCV'] / weekday_shares_2019.loc[row['road_type'],'LCV'], axis = 1)

In [7]:
def calculate_daily_co2_emissions(date):
    diurnal_cycles = cycles.get_hourly_scaling_factors(date=date)
    vehicle_shares = cycles.get_vehicle_share(date=date).to_dict()
    daily_scaling = cycles.get_daily_scaling_factors(date=date).to_dict()

    em_sum_dict = dict()

    for row in visum_red.to_dict('records'):

        # get relevant information from the visum model
        road_link_id = row['road_link_id']
        dtv_visum = row['dtv_SUM']
        road_type = row['road_type']
        hour_capacity = row['hour_capacity']
        visum_speed = row['speed']
        visum_slope = row['SLOPE']
        hgv_corr = row['hgv_corr']
        lcv_corr = row['lcv_corr']
        
        # get vehicle shares from counting data
        hgv_share = vehicle_shares['HGV'][road_type]
        lcv_share = vehicle_shares['LCV'][road_type]
        pc_share = vehicle_shares['PC'][road_type]
        mot_share = vehicle_shares['MOT'][road_type]
        bus_share = vehicle_shares['BUS'][road_type]
        
        # calculate vehicle share correction factor
        k = (1- (hgv_corr * hgv_share)- (lcv_corr * lcv_share)) / (1 - hgv_share - lcv_share)
        
        # calculate vehicle counts and apply correction factor
        dtv = dict()
        dtv_day = dtv_visum * daily_scaling[road_type]
        dtv.update({'HGV' : (dtv_day * hgv_share * hgv_corr)})
        dtv.update({'LCV' : (dtv_day * lcv_share * lcv_corr)})
        dtv.update({'PC' : (dtv_day * pc_share * k)})
        dtv.update({'MOT' : (dtv_day * mot_share * k)})
        dtv.update({'BUS' : (dtv_day * bus_share * k)})
        
        # calculate emissions on the respective road link    
        em = hbefa.calculate_emissions_daily(dtv, diurnal_cycles, road_type,
                                            visum_speed, visum_slope, hour_capacity, 2019)
        
        em_sum_dict.update({road_link_id:em})
    print('Finished '+ date)
    return em_sum_dict

In [8]:
emissions = calculate_daily_co2_emissions(date = '2019-06-02')

Finished 2019-06-02


In [9]:
emissions[33564]['CO2(rep)']

{'PC': 30062.04579329592,
 'LCV': 2222.5526366867243,
 'HGV': 136.2454759394847,
 'BUS': 1331.3181897838904,
 'MOT': 787.8668404464042}